# Neural Collabrative Filtering to be compared with DRR

In [1]:
import os
import time
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter

import model
import config
import evaluate
import data_utils

In [2]:
lr=0.01
dropout=0.0
batch_size=256
epochs=1
top_k=10
factor_num=32
num_layers=3
num_ng=4
test_num_ng=99
out=True

In [3]:
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all()

# construct the train and test datasets
train_dataset = data_utils.NCFData(train_data, item_num, train_mat, num_ng, True)
test_dataset = data_utils.NCFData(test_data, item_num, train_mat, 0, False)
train_loader = data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,batch_size=test_num_ng+1, shuffle=False, num_workers=0)

In [4]:
if config.model == 'NeuMF-pre':
    assert os.path.exists(config.GMF_model_path), 'lack of GMF model'
    assert os.path.exists(config.MLP_model_path), 'lack of MLP model'
    #GMF_model = torch.load(config.GMF_model_path)
    MLP_model = torch.load(config.MLP_model_path)
else:
    GMF_model = None
    MLP_model = None

model = model.NCF(user_num, item_num, factor_num, num_layers, dropout, config.model, GMF_model, MLP_model)

In [5]:
loss_function = nn.BCEWithLogitsLoss()

if config.model == 'NeuMF-pre':
	optimizer = optim.SGD(model.parameters(), lr=lr)
else:
	optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
count, best_hr = 0, 0
for epoch in range(epochs):
    model.train() # Enable dropout (if have).
    start_time = time.time()
    train_loader.dataset.ng_sample()

    for user, item, label in train_loader:
        label = label.float()
        model.zero_grad()
        prediction = model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        # writer.add_scalar('data/loss', loss.item(), count)
        count += 1
    model.eval()
    HR, NDCG = evaluate.metrics(model, test_loader, top_k)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))
    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if out:
            if not os.path.exists(config.model_path):
                os.mkdir(config.model_path)
            torch.save(model, 
                '{}{}.pth'.format(config.model_path, config.model))
print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

The time elapse of epoch 000 is: 00: 18: 46
HR: 0.593	NDCG: 0.334
End. Best epoch 000: HR = 0.593, NDCG = 0.334
